okay! Today we are going to make a twitter project

In [393]:
#import relevent directories
#TODO!
# fix the twitter image
# adjust line
# adjust height of usernames
# add link to box
# set daily time
# improve location finder

In [414]:
#set up helpful stuff I'm using a python twitter API 
import tweepy
import pandas as pd
import datetime

In [496]:
from states import StatesByCode, CodeByStates #import a python files that just has these two variables to make things clearer
from ps import consumer_key, consumer_secret, access_token_secret, sender_email,receiver_email,password_email, access_token, user_name

#sign into twitter
auth = tweepy.OAuth1UserHandler(consumer_key, consumer_secret, access_token, access_token_secret)
api2 = tweepy.API(auth)

In [504]:
# change this if you want to adjust the number of topics or the number of tweets per day
number_of_trends = 5
number_of_tweets = 5
preferred_location = '' # this needs to be City, State e.g. New York City, New York

In [498]:
if preferred_location == '':
    #start with getting the user information -> extract the location possible
    user= api2.get_user(screen_name = user_name)
    profile_location = user.location
    [cityplus, state] = profile_location.split()
    if len(state)> 2:
        st_ab = CodeByStates[state]
    else:
        st_ab = state
    city = cityplus.strip(',')
    my_places = api2.search_geo(query = city)

    #I happen to know davis, CA is the first one so I'm going to this short hand for now
    preferred_location = my_places[0]
    
#certain places like small towns etc won't have an active trending area. So here we are going to find the closest WOEID that twitter cares about
center = this_place.centroid
closest = api2.closest_trends(center[1],center[0])
closest_woeid = closest[0]['woeid']


In [515]:
#here I actually get the trends for the location
trends = api2.get_place_trends(id = closest_woeid)[0]['trends'][0:number_of_trends]

all_tweets=[] #this is going to be what we actually care about in the end

for trend in trends:
    indv_trends = api2.search_tweets(trend['query']) #search for the tweets related to the trend
    #I want to reorder the tweets based on how trending each tweet is
    counts = [x.retweet_count + x.favorite_count for x in indv_trends]
    order = []
    for x in range(len(counts)):
        loc = counts.index(max(counts))
        order.append(loc)
        counts[loc] = -1
    indv_trends_reord = [indv_trends[x] for x in order]
    #now extract the top 5 tweets 
    spec_tweets = getSpecTweets(indv_trends_reord, number_of_tweets)
    #add the topic, tweets, and query url to all_tweets
    all_tweets.append([trend['query'],spec_tweets,trend['url']])
    


In [439]:

#allbox is where we will put our entire message
allbox = """ <html>
<head>

</head>

   <body>
   <div style="border-radius:15px;-webkit-border-radius:10px;-moz-border-radius:15px;width:525 px;padding-left:9px;padding-top:6px; 
                  padding-bottom:6px;margin:2px;
               background-color:#EFFEFE ;margin-left:0;padding:10px;">
   <p style="color:3370AA;font-size:20.0pt;">twitter topics<br>
    <br> """


for trend in all_tweets:
    #sometimes I see weird things in front of the trends or a + instead of space so I'm replacing them here and adding them to the message
    if '%22' in trend[0]:
        trend[0] = trend[0].replace('%22','')
    if '%23' in trend[0]:
        trend[0] = trend[0].replace('%23', '')
    elif '+' in trend[0]:
        trend[0] = trend[0].replace('+', ' ')
    the_link = trend[2]
    allbox += """ <a href= """ + the_link 
    allbox += """ > <span style="color:black;font-size:30.0pt;text-decoration:none"> #""" + trend[0] + """ </span> </a> """
    #allbox += """ <span style="color:black;font-size:30.0pt"> #""" + trend[0] + """ </span><br><br> """
    allbox += """ <hr width="50%" size="5" align="left" color="7C94AA" noshade> """
    
    #this is going to loop and generate each box with the tweets using the addtweet function
    for twetz in trend[1]: 
        profile_image = twetz.user.profile_image_url
        tweet_body = twetz.text
        the_user_name = twetz.user.screen_name
        allbox = allbox +addtweet(profile_image, tweet_body, the_user_name)
    allbox += """ <br> <br> """

allbox += """     </div ></p> </body> </html> """  

In [513]:
sendemail(sender_email, receiver_email, password_email, allbox)

In [512]:
def sendemail(sender_email, receiver_email, password, allbox):
    #sendemail(string, string, string, string) -> nothing
    # email address, email address, email password, text of the message
    import smtplib, ssl
    from email.mime.text import MIMEText
    from email.mime.multipart import MIMEMultipart

    message = MIMEMultipart("alternative")
    message["Subject"] = "Twitter Daily Update"
    message["From"] = sender_email
    message["To"] = receiver_email


    # Turn these into plain/html MIMEText objects
    part = MIMEText(allbox, "html")

    # Add HTML/plain-text parts to MIMEMultipart message
    # The email client will try to render the last part first

    message.attach(part)

    # Create secure connection with server and send email
    context = ssl.create_default_context()
    with smtplib.SMTP_SSL("smtp.gmail.com", 465, context=context) as server:
        server.login(sender_email, password)
        server.sendmail(
            sender_email, receiver_email, message.as_string()
        )

In [514]:
def addtweet(profile_image, tweet_body, user_name):
    #addtweet(string, string, string) -> string
    #takes the location of the tweeters profile picture, the tweet, and the username and generates the tweet image to be added to the email
    addtext = """ <div style="border-radius:15px;-webkit-border-radius:15px;-moz-border-radius:15px;width:500px;padding-left:9px;padding-top:6px; 
              padding-bottom:6px;margin:2px; background-color:#82cfd9;margin-left:0;padding:10px;">

               <div <a href= "https://twitter.com/home" style="border-radius:15px;-webkit-border-radius:10px;-moz-border-radius:15px;width:480px;padding-left:9px;padding-top:6px; 
                  padding-bottom:6px;margin:2px;
               background-color:#ebfffe;margin-left:0;padding:10px;">

                  <img src="images/twitter_logo.png" style="width:50px;height:50px;" />
                  <img src= """ + profile_image + """ style="width:50px;height:50px;" />
                   <span style="color:black;font-size:12.0pt"> """ + '@' + user_name + """ <span>

                    </a>
                    <p> <span style="color:black;font-size:10.0pt"> """ + tweet_body + """ </span> </p>
              </div>
          </div> """
    return addtext

In [508]:
def getSpecTweets(each_trend, number_of_tweets):
    #getSpecTweets[tweepy.models.SearchResults, int] -> list
    #takes a given trend and number of tweets you want from that trend and returns a list that number of unique tweets
    startingText = []
    spec_trends = []
    for tweet in each_trend:
        tweet_text = tweet.text
        #I want to make sure that the same tweet retweeted or a bot doesn't take up a ton of the tweets
        if any(tweet_text[0:25]  in x  for x in startingText):
            continue
        else:
            startingText.append(tweet_text)
            spec_trends.append(tweet)
        if len(spec_trends) == number_of_tweets:
            break
    return spec_trends

In [ ]:
#okay lets get some regional information
#lets load in this pdf I have of clean woeid data I got off https://www.woeids.com/index.html
df = pd.read_csv('geo_clean.csv') #it is a really big csv (1,000,000+ rows!) so if you are only interested in the US you can instead use reduced_geo_clean.csv

my_woeid_tab = df[(df.area == county) & (df.locality == city) & (df.country == 'US')]
if l.shape[0] == 0:
    print('wait! the location was not located. Try a different area or country')

my_woeid = my_woeid_tab['woeid'].tolist()[0]
print(my_)